# Using GLM-4.5 and setting up multi role division of labor to complete complex tasks

This code establishes a simple customer service prompt scenario. Through different prompts, the model is equipped to professionally respond within various sub-scenarios. The code aims to demonstrate how to use the GLM model to achieve:

1. Thought process for scheduling sub-task prompts within specific scenarios.
2. Intent recognition and switching between different scenarios for the large model.
3. Design thinking for multi-role division of labor within scenarios.
4. Completing more complex tasks by using multiple roles with the same base model.

Please note that this repository does not involve the Function Call or Plan stages of the Agent. It only covers simple role switching.

首先，将 API Key 信息填入系统环境中。

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

I have set up four roles for the current scenario:
1. General Customer Service: This customer service will simply identify user questions and assign them to professional staff to complete
2. Registration Staff: This staff member understands how to handle user registration issues, including what information needs to be obtained from users, and will return registration results to users.
3. Deregistration Staff: This staff member understands how to handle user deregistration issues, including what information needs to be obtained from users, and tells users how the results are provided
4. Query Staff: This staff member understands how to handle user query issues, including obtaining the correct query content from users and returning query results.

Here, we have designed prompts for the scenario.

Since GLM has strong instruction-following capabilities, I added special fields in intent recognition. Different roles are switched through matching special fields. In actual engineering, the situation is far more complex than this.

In [2]:
sys_prompt = """你是一个聪明的客服。您将能够根据用户的问题将不同的任务分配给不同的人。您有以下业务线：
1.用户注册。如果用户想要执行这样的操作，您应该发送一个带有"registered workers"的特殊令牌。并告诉用户您正在调用它。
2.用户数据查询。如果用户想要执行这样的操作，您应该发送一个带有"query workers"的特殊令牌。并告诉用户您正在调用它。
3.删除用户数据。如果用户想执行这种类型的操作，您应该发送一个带有"delete workers"的特殊令牌。并告诉用户您正在调用它。
"""
registered_prompt = """
您的任务是根据用户信息存储数据。您需要从用户那里获得以下信息：
1.用户名、性别、年龄
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，您需要提示用户提供。如果用户提供了此信息，则需要将此信息存储在数据库中，并告诉用户注册成功。
存储方法是使用SQL语句。您可以使用SQL编写插入语句，并且需要生成用户ID并将其返回给用户。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
query_prompt = """
您的任务是查询用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
如果用户没有提供此信息，则需要提示用户提供。如果用户提供了此信息，那么需要查询数据库。如果用户ID和密码匹配，则需要返回用户的信息。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""
delete_prompt = """
您的任务是删除用户信息。您需要从用户那里获得以下信息：
1.用户ID
2.用户设置的密码
3.用户的电子邮件地址
如果用户没有提供此信息，则需要提示用户提供该信息。
如果用户提供了这些信息，则需要查询数据库。如果用户ID和密码匹配，您需要通知用户验证码已发送到他们的电子邮件，需要进行验证。
如果用户没有新问题，您应该回复带有 "customer service" 的特殊令牌，以结束任务。
"""

Next, we designed a simple class with the following functions:
1. We separate the history of each different role. These histories do not interact with each other, but will pass user requests or model responses to another history when needed.
2. Implement simple continuous conversation, and add intent recognition (prompt setting) when acting as customer service, and conduct simple multi-turn conversations in the case of professional staff.

In [3]:
class SmartAssistant:
    def __init__(self):
        self.client = ZhipuAI()

        self.system_prompt = sys_prompt
        self.registered_prompt = registered_prompt
        self.query_prompt = query_prompt
        self.delete_prompt = delete_prompt

        # Using a dictionary to store different sets of messages
        self.messages = {
            "system": [{"role": "system", "content": self.system_prompt}],
            "registered": [{"role": "system", "content": self.registered_prompt}],
            "query": [{"role": "system", "content": self.query_prompt}],
            "delete": [{"role": "system", "content": self.delete_prompt}]
        }

        # Current assignment for handling messages
        self.current_assignment = "system"

    def get_response(self, user_input):
        self.messages[self.current_assignment].append({"role": "user", "content": user_input})
        while True:
            response = self.client.chat.completions.create(
                model="glm-4.5",
                messages=self.messages[self.current_assignment],
                temperature=0.9,
                stream=False,
                max_tokens=8192,
            )

            ai_response = response.choices[0].message.content
            if "registered workers" in ai_response:
                self.current_assignment = "registered"
                print("意图识别:",ai_response)
                print("switch to <registered>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "query workers" in ai_response:
                self.current_assignment = "query"
                print("意图识别:",ai_response)
                print("switch to <query>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "delete workers" in ai_response:
                self.current_assignment = "delete"
                print("意图识别:",ai_response)
                print("switch to <delete>")
                self.messages[self.current_assignment].append({"role": "user", "content": user_input})
            elif "customer service" in ai_response:
                print("意图识别:",ai_response)
                print("switch to <customer service>")
                self.messages["system"] += self.messages[self.current_assignment]
                self.current_assignment = "system"
                return ai_response
            else:
                self.messages[self.current_assignment].append({"role": "assistant", "content": ai_response})
                return ai_response

    def start_conversation(self):
        while True:
            user_input = input("User: ")
            if user_input.lower() in ['exit', 'quit']:
                print("Exiting conversation.")
                break
            response = self.get_response(user_input)
            print("Assistant:", response)

Next, we can start continuous conversation. The model will print the model's thinking and response process, and will print whether roles have been switched.

In [4]:
assistant = SmartAssistant()
assistant.start_conversation()

意图识别: 我理解您想要注册。我正在为您调用注册服务。

registered workers
switch to <registered>
Assistant: 您好！我可以帮您注册。为了完成注册，我需要收集以下信息：

1. 基本信息：
   - 用户名
   - 性别
   - 年龄

2. 安全信息：
   - 您想设置的密码

3. 联系信息：
   - 电子邮件地址

请提供这些信息，我将为您完成注册。
Assistant: 
感谢您提供基本信息！我已记录：
- 用户名：zR
- 性别：男
- 年龄：18岁

为了完成注册，我还需要您提供以下信息：
1. 您想设置的密码
2. 您的电子邮件地址

请提供这些信息，以便我为您完成注册。
意图识别: 
感谢您提供所有必要信息！我现在将为您完成注册。

我已将您的信息存储在数据库中，以下是执行的SQL语句：
```sql
INSERT INTO users (username, gender, age, password, email) 
VALUES ('zR', '男', 18, '123456', 'zR@qq.com');
```

注册成功！您的用户ID是：1001

请妥善保管您的用户ID和密码，以便日后登录使用。如果您有任何其他问题或需要帮助，请随时告诉我。

customer service
switch to <customer service>
Assistant: 
感谢您提供所有必要信息！我现在将为您完成注册。

我已将您的信息存储在数据库中，以下是执行的SQL语句：
```sql
INSERT INTO users (username, gender, age, password, email) 
VALUES ('zR', '男', 18, '123456', 'zR@qq.com');
```

注册成功！您的用户ID是：1001

请妥善保管您的用户ID和密码，以便日后登录使用。如果您有任何其他问题或需要帮助，请随时告诉我。

customer service


KeyboardInterrupt: Interrupted by user

## Reminder

Here, I only used the simplest switching method to switch roles. In actual engineering, you should pay attention to these issues:
1. Engineering construction in role switching
2. History management
3. This code does not involve tool calling. Otherwise, you would also need to design corresponding tools and Function Call functionality for different roles.